# Load csv file from URL

In [1]:
import psycopg2
import csv
import io
from urllib import request
from datetime import datetime
import pandas as pd

response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))
header = next(reader)
storm_data = [row for row in reader]

print(storm_data[:5])

[['2001', '1957', '8', '8', '1800Z', '63', 'NOTNAMED', '22.5', '-140', '50', '0', 'TS', 'Eastern Pacific', '1.140175'], ['2002', '1961', '10', '3', '1200Z', '116', 'PAULINE', '22.1', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.16619'], ['2003', '1962', '8', '29', '0600Z', '124', 'C', '18', '-140', '45', '0', 'TS', 'Eastern Pacific', '2.10238'], ['2004', '1967', '7', '14', '0600Z', '168', 'DENISE', '16.6', '-139.5', '45', '0', 'TS', 'Eastern Pacific', '2.12132'], ['2005', '1972', '8', '16', '1200Z', '251', 'DIANA', '18.5', '-139.8', '70', '0', 'H1', 'Eastern Pacific', '1.702939']]


# Create users with specific privileges

In [2]:
conn = psycopg2.connect('dbname=postgres user=postgres password=tdi')
cur = conn.cursor()


# Create Data Production user that can insert, update, and read the data but not delete
cur.execute('DROP ROLE IF EXISTS dbAdmin')
cur.execute('''CREATE USER dbAdmin WITH PASSWORD 'IHW' NOSUPERUSER CREATEDB;
                REVOKE ALL ON storm FROM dbAdmin;
                GRANT INSERT, UPDATE, SELECT ON storm TO dbAdmin''')

# Create Analyst group
cur.execute('DROP ROLE IF EXISTS analyst')
cur.execute('''CREATE GROUP analyst WITH NOLOGIN;
                REVOKE ALL ON storm FROM analyst;
                GRANT SELECT ON storm TO analyst;''')

conn.commit()

# Determine data types

In [3]:
response2 = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
stormy = pd.read_csv(io.TextIOWrapper(response2))

print(stormy.astype('str').applymap(lambda x: len(x)).max())

ï»¿FID         5
YEAR           4
MONTH          2
DAY            2
AD_TIME        5
BTID           4
NAME           9
LAT            4
LONG           7
WIND_KTS       3
PRESSURE       4
CAT            2
BASIN         15
Shape_Leng    19
dtype: int64


# Convert AD_Time value into HH:MM format

In [4]:
for row in storm_data:
    row[4] = datetime.strptime(row[4][0:2] + ":" + row[4][2:4], "%H:%M").time()

# Create new table

New table 'storm' was created and loaded with values from csv file

In [5]:
cur.execute('DROP TABLE IF EXISTS storm')
cur.execute('''
    CREATE TABLE storm (
        FID integer PRIMARY KEY, 
        YEAR integer, 
        MONTH integer, 
        DAY integer,
        AD_TIME time,
        BTID integer,
        NAME varchar(9),
        LAT float,
        LONG float,
        WIND_KTS float, 
        PRESSURE float,
        CAT varchar(2), 
        BASIN varchar(15),
        Shape_Leng decimal(8,6))''')

# Insert value into table 
for row in storm_data:
    cur.execute('INSERT INTO storm VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', row)

# Add date column with timestamp format

In [6]:
cur.execute('ALTER TABLE storm ADD DATE TIMESTAMP')
cur.execute("UPDATE storm SET DATE = to_timestamp(YEAR || '-' || MONTH || '-' || DAY || ' ' || AD_TIME, 'YYYY-MM-DD HH24:MI:SS')")

conn.commit()

# Remove extra columns

In [7]:
cur.execute("ALTER TABLE storm DROP COLUMN YEAR, DROP MONTH, DROP DAY, DROP AD_TIME")
conn.commit()

In [11]:
sql = "SELECT * FROM storm ORDER BY date"
df = pd.read_sql(sql, conn)
df

#cur.execute(sql)
#li = cur.fetchall()


,fid,btid,name,lat,long,wind_kts,pressure,cat,basin,shape_leng,date
0,55291,1,NOTNAMED,28.0,-94.8,80.0,0.0,H1,North Atlantic,0.600000,1851-06-25 00:00:00
1,55560,1,NOTNAMED,28.0,-95.4,80.0,0.0,H1,North Atlantic,0.600000,1851-06-25 06:00:00
2,55567,1,NOTNAMED,28.0,-96.0,80.0,0.0,H1,North Atlantic,0.509902,1851-06-25 12:00:00
3,55573,1,NOTNAMED,28.1,-96.5,80.0,0.0,H1,North Atlantic,0.509902,1851-06-25 18:00:00
4,55580,1,NOTNAMED,28.2,-97.0,70.0,0.0,H1,North Atlantic,0.608276,1851-06-26 00:00:00
5,55585,1,NOTNAMED,28.3,-97.6,60.0,0.0,TS,North Atlantic,0.707107,1851-06-26 06:00:00
6,56459,1,NOTNAMED,28.4,-98.3,60.0,0.0,TS,North Atlantic,0.632456,1851-06-26 12:00:00
7,56461,1,NOTNAMED,28.6,-98.9,50.0,0.0,TS,North Atlantic,0.640312,1851-06-26 18:00:00
8,56464,1,NOTNAMED,29.0,-99.4,50.0,0.0,TS,North Atlantic,0.640312,1851-06-27 00:00:00
9,56467,1,NOTNAMED,29.5,-99.8,40.0,0.0,TS,North Atlantic,0.538516,1851-06-27 06:00:00
